d
Maggy enables you to train with Tensorflow distributed optimizers.
Using Maggy, you have to make minimal changes in train your model in a distributed fashion.

### 0. Spark Session

Make sure you have a running Spark Session/Context available.

On Hopsworks, just run your notebook to start the spark application.

### 1. Model definition

Let's define the model we want to train. The layers of the model have to be defined in the \_\_init__ function.

Do not instantiate the class, otherwise you won't be able to use Maggy.

In [0]:
from pyspark.sql import SparkSession
import os

try:
  import tensorflow as tf
except ModuleNotFoundError:
  %pip install tensorflow-cpu==2.4.1


In [0]:
import tensorflow as tf
if tf.__version__ != '2.4.1':
  %pip install tensorflow-cpu==2.4.1

Python interpreter will be restarted.
Collecting tensorflow-cpu==2.4.1
 Downloading tensorflow_cpu-2.4.1-cp37-cp37m-manylinux2010_x86_64.whl (144.1 MB)
Collecting flatbuffers~=1.12.0
 Downloading flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
Requirement already satisfied: google-pasta~=0.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorflow-cpu==2.4.1) (0.2.0)
Collecting six~=1.15.0
 Downloading six-1.15.0-py2.py3-none-any.whl (10 kB)
Requirement already satisfied: opt-einsum~=3.3.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorflow-cpu==2.4.1) (3.3.0)
Collecting numpy~=1.19.2
 Downloading numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
Collecting tensorboard~=2.4
 Downloading tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
Collecting tensorflow-estimator<2.5.0,>=2.4.0
 Downloading tensorflow_estimator-2.4.0-py2.py3-none-any.whl (462 kB)
Requirement already satisfied: termcolor~=1.1.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorflow-cpu==2.4.1) (1.1.0)
Collecting grpcio~=1.32.0
 Downloading grpcio-1.32.0-cp37-cp37m-manylinux2014_x86_64.whl (3.8 MB)
Requirement already satisfied: typing-extensions~=3.7.4 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorflow-cpu==2.4.1) (3.7.4.3)
Collecting wheel~=0.35
 Downloading wheel-0.36.2-py2.py3-none-any.whl (35 kB)
Requirement already satisfied: protobuf>=3.9.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorflow-cpu==2.4.1) (3.11.4)
Requirement already satisfied: h5py~=2.10.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorflow-cpu==2.4.1) (2.10.0)
Requirement already satisfied: astunparse~=1.6.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorflow-cpu==2.4.1) (1.6.3)
Collecting wrapt~=1.12.1
 Downloading wrapt-1.12.1.tar.gz (27 kB)
Collecting absl-py~=0.10
 Downloading absl_py-0.12.0-py3-none-any.whl (129 kB)
Requirement already satisfied: gast==0.3.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorflow-cpu==2.4.1) (0.3.3)
Requirement already satisfied: keras-preprocessing~=1.1.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorflow-cpu==2.4.1) (1.1.2)
Collecting tensorboard-data-server<0.7.0,>=0.6.0
 Downloading tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
Requirement already satisfied: google-auth-oauthlib<0.5,>=0.4.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.1) (0.4.1)
Requirement already satisfied: tensorboard-plugin-wit>=1.6.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.1) (1.7.0)
Requirement already satisfied: requests<3,>=2.21.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.1) (2.22.0)
Requirement already satisfied: markdown>=2.6.8 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.1) (3.1.1)
Requirement already satisfied: setuptools>=41.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-54e0165f-adde-4c2b-970a-ef83f9b4563a/lib/python3.7/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.1) (45.2.0.post20200210)
Requirement already satisfied: 

In [0]:
from tensorflow import keras 
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

# you can use keras.Sequential(), you just need to override it 
# on a custom class and define the layers in __init__()
class NeuralNetwork(Sequential):
        
    def __init__(self, nl=4):
        
        super().__init__()
        self.add(Dense(10,input_shape=(None,4),activation='tanh'))
        if nl >= 4:
          for i in range(0, nl-2):
            self.add(Dense(8,activation='tanh'))
        self.add(Dense(3,activation='softmax'))

model = NeuralNetwork

### 2. Dataset creation

You can create the dataset here and pass it to the TfDistributedConfig, or creating it in the training function.

You need to change the dataset path is correct.

In [0]:
display(dbutils.fs.ls("/FileStore/tables/iris_train-2.csv"))

path,name,size
dbfs:/FileStore/tables/iris_train-2.csv,iris_train-2.csv,3745


In [0]:
train_set_path = "dbfs:/FileStore/tables/iris_train-2.csv"
test_set_path = "dbfs:/FileStore/tables/iris_test-1.csv"

In [0]:
train_set = spark.read.format("csv").option("header","true")\
  .option("inferSchema", "true").load(train_set_path).drop('_c0')

test_set = spark.read.format("csv").option("header","true")\
  .option("inferSchema", "true").load(test_set_path).drop('_c0')

raw_train_set = train_set.toPandas().values
raw_test_set = test_set.toPandas().values

In [0]:
def process_data(train_set, test_set):
    
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    
    X_train = train_set[:,0:4]
    y_train = train_set[:,4:]
    X_test = test_set[:,0:4]
    y_test = test_set[:,4:]

    return (X_train, y_train), (X_test, y_test)
  
train_set, test_set = process_data(raw_train_set, raw_test_set)

### 3. Defining the training function

The programming model is that you wrap the code containing the model training inside a wrapper function. Inside that wrapper function provide all imports and parts that make up your experiment.

The function should return the metric that you want to optimize for. This should coincide with the metric being reported in the Keras callback (see next point).
You can return the metric list, in this case only the loss element will be printed.

In [0]:
def hpo_function(number_layers, reporter):
  
  model = NeuralNetwork(nl=number_layers)
  model.build()
  
  #fitting the model and predicting
  model.compile(Adam(lr=0.04),'categorical_crossentropy',metrics=['accuracy'])
  train_input, test_input = process_data(raw_train_set, raw_test_set)

  train_batch_size = 75
  test_batch_size = 15
  epochs = 10
  
  model.fit(x=train_input[0], y=train_input[1],
            batch_size=train_batch_size,
            epochs=epochs,
            verbose=1)

  score = model.evaluate(x=test_input[0], y=test_input[1], batch_size=test_batch_size, verbose=1)
                         
  print(f'Test loss: {score[0]}')
  print(f'Test accuracy: {score[1]}')

  return score[1]

In [0]:
def training_function(model, train_set, test_set, hparams):
    
    model = model(nl=hparams['number_layers'])
    model.build()
    #fitting the model and predicting

    model.compile(Adam(lr=hparams['learning_rate']),'categorical_crossentropy',metrics=['accuracy'])
    
    #raise ValueError(list(train_set.as_numpy_iterator()))

    model.fit(train_set,epochs=hparams['epochs'])

    accuracy = model.evaluate(test_set)

    return accuracy

### 4. Configuring the experiment

In order to use maggy distributed training, we have to configure the training model, we can pass it to TfDistributedConfig.
the model class has to be an implementation of __tf.keras.Model__.
We can also define __train_set__, __test_set__ and eventually the __model_parameters__. __model_parameters__ is a dictionary
containing the parameters to be used in the \_\_init__ function of your model.

In [0]:
from maggy.experiment_config import OptimizationConfig
from maggy import Searchspace

# The searchspace can be instantiated with parameters
sp = Searchspace(number_layers=('INTEGER', [2, 8]))

hpo_config = OptimizationConfig(num_trials=4, optimizer="randomsearch", searchspace=sp, direction="max", es_interval=1, es_min=5, name="hp_tuning_test")

Hyperparameter added: number_layers

### 5. Run distributed training

Finally, we are ready to launch the maggy experiment. You just need to pass 2 parameters: the training function and the configuration variable we defined in the previous steps.

In [0]:
from maggy import experiment

result = experiment.lagom(train_fn=hpo_function, config=hpo_config)

print(result)

------ RandomSearch Results ------ direction(max) 
BEST combination {"number_layers": 3} -- metric 1.0
WORST combination {"number_layers": 7} -- metric 0.5666666626930237
AVERAGE metric -- 0.8583333194255829
EARLY STOPPED Trials -- 0
Total job time 0 hours, 0 minutes, 15 seconds

Finished experiment.
{'best_id': '0a1e587da4ba033b', 'best_val': 1.0, 'best_config': {'number_layers': 3}, 'worst_id': '882491b7850528b2', 'worst_val': 0.5666666626930237, 'worst_config': {'number_layers': 7}, 'avg': 0.8583333194255829, 'metric_list': [0.8999999761581421, 0.5666666626930237, 1.0, 0.9666666388511658], 'num_trials': 4, 'early_stopped': 0, 'num_epochs': 0, 'trial_id': '0a1e587da4ba033b'}

In [0]:
from maggy.experiment_config.tf_distributed import TfDistributedConfig

#define the constructor parameters of your model
model_params = {
    #train dataset entries / num_workers
    'train_batch_size': 75,
    #test dataset entries / num_workers
    'test_batch_size': 15,
    'learning_rate': 0.04,
    'epochs': 20,
    'number_layers': result['best_config']['number_layers'],
}

training_config = TfDistributedConfig(name="tf_test", model=model, train_set=train_set, test_set=test_set, process_data=process_data, hparams = model_params)

In [0]:
experiment.lagom(training_function, training_config)

Final average test loss: 0.178
Finished experiment. Total run time: 0 hours, 0 minutes, 16 seconds
Out[17]: {'test result': 0.17838226817548275}